In [1]:
#데이터 로드
import pandas as pd 
import numpy as np

# Data

In [2]:
#기본데이터
manu_val = pd.read_csv("Z:/team/dataset/제조업 부가가치(분기별).csv")
month_supply= pd.read_csv("Z:/team/dataset/월별공급량및비중.csv")
ratio_price = pd.read_csv("Z:/team/dataset/상업용 상대가격(기준=2015).csv")
sample_submission = pd.read_csv("Z:/team/dataset/submission_sample.csv")

#기온데이터 추가
#월별로 가져온 외부데이터
temperature= pd.read_csv('Z:/team/extra_data/전처리 전 원본/월별)기온_1995-2022.csv',header=6, encoding='cp949')
rain= pd.read_csv('Z:/team/extra_data/전처리 전 원본/월별)강수량_1995-2022.csv',header=6,encoding='cp949')
cpi= pd.read_csv('Z:/team/extra_data/월별/월별)소비자물가지수1995-2022.csv', encoding='cp949')
gdp = pd.read_csv("Z:/team/extra_data/분기별)경제활동별_GDP_및_GNI_계절조정_1995-2022.csv", encoding='cp949')
gas_income= pd.read_csv('Z:/team/extra_data/월별/월별)한국의 대륙별 천연가스 수입 현황_1988-2022.csv', encoding='cp949')

month_supply_ratio_price = month_supply.merge(ratio_price)

#분기 표현
def quarter(month):
    q = ''
    if month <= 3: q = 'Q1'
    elif month <=6: q = 'Q2'
    elif month <=9: q= 'Q3'
    else: q='Q4'
    return q

month_supply_ratio_price['QUARTER']=month_supply_ratio_price['MONTH'].apply(lambda x:quarter(x))

#최종 병합 데이터
df = month_supply_ratio_price.merge(manu_val)

In [7]:
df.isnull().sum()

YEAR                   0
MONTH                  0
도시가스(톤)_민수용            0
도시가스(톤)_산업용            0
도시가스(톤)_총합(민수용+산업용)    0
민수용비중                  0
산업용비중                  0
RP(상대가격)               0
GAS_PRICE(산업용도시가스)     0
OIL_PRICE(원유정제처리제품)    0
QUARTER                0
QVA(제조업부가가치/단위:십억원)    0
dtype: int64

In [8]:
df.shape

(300, 12)

In [10]:
#행 열 Transpose
cpi_t = cpi.transpose()
cpi_t = cpi_t.rename(columns=cpi_t.iloc[0])
cpi_t = cpi_t.drop(cpi_t.index[0])
#cpi_t.head(2)

In [11]:
#외부데이터 병합
#원하는 범위를 잘라서 준비(1996-2020 데이터만 이용)
temperature_1996 = temperature.iloc[12:len(temperature)-21]
rain_1996 = rain.iloc[12:len(rain)-21]
cpi_t_1996 = cpi_t.iloc[12:len(cpi_t)-20]
gas_income_1996 = gas_income.iloc[96:len(gas_income)-18]

print(len(temperature_1996))
print(len(rain_1996))
print(len(cpi_t_1996))
print(len(gas_income_1996))

300
300
300
300


In [12]:
#제공 데이터와 합치기 위해 인덱스 재설정
temperature_1996.reset_index(inplace=True)
temperature_1996 = temperature_1996.iloc[:,1:]

rain_1996.reset_index(inplace=True)
rain_1996 = rain_1996.iloc[:,1:]

cpi_t_1996.reset_index(inplace=True)
#cpi_t_1996 = cpi_t_1996.iloc[:,1:]

#중량총합
gas_income_1996.reset_index(inplace=True)
gas_1996_weight = gas_income_1996.iloc[:,[5,8,11,14, 17, 20,23, 26,29]]
gas_1996_weight['천연가스 수입 중량 합계'] = gas_1996_weight.sum(axis =1)


C:\Users\kogas\AppData\Local\Temp\3\ipykernel_8648\4260283601.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gas_1996_weight['천연가스 수입 중량 합계'] = gas_1996_weight.sum(axis =1)


In [13]:
#컬럼명에 단위 없애고 재설정
temperature_1996.set_axis(["년월","지점","평균기온","최저기온","최고기온"], axis=1, inplace=True)
rain_1996.set_axis(["년월","지점","강수량"], axis=1, inplace=True)
cpi_t_1996.set_axis(["년월", "소비자 물가지수(cpi)"], axis=1, inplace=True)

In [14]:
#원하는 컬럼만 추출
temperature_1996_df = pd.DataFrame(temperature_1996.iloc[:,2:6])
rain_1996_df = pd.DataFrame(rain_1996.iloc[:,2])
cpi_t_1996_df = pd.DataFrame(cpi_t_1996.iloc[:,1])
gas_1996_weight_df = pd.DataFrame(gas_1996_weight.iloc[:,9])

In [15]:
#gdp
gdp = pd.DataFrame(gdp.loc[47])
gdp = gdp.rename(columns=gdp.iloc[0])
gdp = gdp.drop(gdp.index[0])

In [16]:
#원하는 범위를 잘라서 준비(1996-2020 데이터만 이용)
gdp_1996_2020 = gdp.iloc[4:len(gdp)-6]

#1개를 3개로 복사해줌(분기->월)
df_list=[]
for i in range(len(gdp_1996_2020)):
    if i != len(gdp_1996_2020):
        df_temp = gdp_1996_2020[i:i+1]
        d = df_temp.iloc[0,0]
        df_list.append(df_temp)
        
        to_month = pd.DataFrame(index = range(0,2), columns=gdp_1996_2020.columns, data=[d,d])
        df_list.append(to_month)
gdp_in_month = pd.concat(df_list).reset_index(drop=True)

In [17]:
#인덱스 재설정
year_month=[]
for i in range(25):
    year=1996+i
    for m in range(1,13):
        month=m
        year_month.append(str(year) + "." + str(month))
print(len(year_month))
gdp_in_month.set_axis(year_month, axis=0, inplace=True)

300


In [18]:
gdp_in_month.reset_index(inplace=True)
gdp_in_month.set_axis(["년월", "국내총생산(시장가격, GDP)"], axis=1, inplace=True)

In [19]:
gdp_in_month_df = pd.DataFrame(gdp_in_month.iloc[:,1])

In [20]:
#기존 데이터와 병합
data_total = pd.concat([df,temperature_1996_df,rain_1996_df,cpi_t_1996_df,gdp_in_month_df,gas_1996_weight_df], axis=1)
#data_total.head(12)
data_total.head(3)

,YEAR,MONTH,도시가스(톤)_민수용,도시가스(톤)_산업용,도시가스(톤)_총합(민수용+산업용),민수용비중,산업용비중,RP(상대가격),GAS_PRICE(산업용도시가스),OIL_PRICE(원유정제처리제품),QUARTER,QVA(제조업부가가치/단위:십억원),평균기온,최저기온,최고기온,강수량,소비자 물가지수(cpi),"국내총생산(시장가격, GDP)",천연가스 수입 중량 합계
0,1996,1,605519.0,83809.0,689328.0,0.87842,0.12158,0.97,26.94,27.86,Q1,36550.3,-1.3,-6.3,4.5,29.5,53.289,118011.1,969744
1,1996,2,566323.0,70427.0,636750.0,0.88940,0.11060,0.93,26.94,29.04,Q1,36550.3,-1.1,-6.4,5.2,11.4,53.499,118011.1,859127
2,1996,3,477514.0,62652.0,540166.0,0.88401,0.11599,0.96,26.94,27.99,Q1,36550.3,4.4,-0.7,10.0,121.2,53.862,118011.1,967455


In [21]:
#기온데이터 추출
#일단 현재까지 붙인 데이터 다시 추출
data_total.insert(0,'Unnamed: 0',pd.date_range(start='1996-01-01', freq='MS',end='2020-12-01'))
data_total.to_csv("Z:/team/dataset/제공데이터_merge+기온,강수량,cpi,gdp,천연가스수입.csv",index=False)